# Data preprocessing

In [48]:
import csv
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score, explained_variance_score

data = []
scaler = MinMaxScaler()

def create_dataset(dataset, look_back=12):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        aux = dataset[i:(i+look_back)]
        dataX.append(aux)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

with open('./datasets/traffic-prediction-dataset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        row_to_append = [float(x) for x in row[0:4]]
        data.append(row_to_append)

scaler.fit(data)
normalized_data = scaler.transform(data)

# Train and Test Sets

In [49]:
train_size = int(len(normalized_data) * 0.75)
test_size = len(normalized_data) - train_size

train, test = normalized_data[0:train_size,:], normalized_data[train_size:len(normalized_data),:]
x_train, y_train = create_dataset(train)
x_test, y_test = create_dataset(test)

# Recurrent Neural Networks

## Training LSTM-NN

In [ ]:
from keras import Input
from keras.models import Sequential 
from keras.layers import Dense, Dropout, InputLayer
from keras.layers import LSTM

model = Sequential()
model.add(InputLayer(input_shape=(12, 4)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

model.fit(x_train, y_train, epochs=50, batch_size=128)

## Testing LSTM-NN

In [ ]:
y_pred = model.predict(x_test)

final_y_real = scaler.inverse_transform(y_test)
final_y_pred = scaler.inverse_transform(y_pred)
mean_absolute_error(final_y_real, final_y_pred)
mean_absolute_percentage_error(final_y_real, final_y_pred) ## fix y_real values equal to 0
mean_squared_error(final_y_real, final_y_pred, squared=False)
r2_score(final_y_real, final_y_pred)
explained_variance_score(final_y_real, final_y_pred)

## Training GRU-NN

In [ ]:
from keras.layers import GRU

gru_model = Sequential()
gru_model.add(InputLayer(input_shape=(12, 4)))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(GRU(64))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(4))
gru_model.compile(loss='mean_squared_error', optimizer='rmsprop')
gru_model.summary()

gru_model.fit(x_train, y_train, epochs=50, batch_size=128)

## Testing GRU-NN

In [ ]:
y_pred_gru = gru_model.predict(x_test)

final_y_real_gru = scaler.inverse_transform(y_test)
final_y_pred_gru = scaler.inverse_transform(y_pred_gru)
mean_absolute_error(final_y_real_gru, final_y_pred_gru)
mean_absolute_percentage_error(final_y_real_gru, final_y_pred_gru) ## fix y_real values equal to 0
mean_squared_error(final_y_real_gru, final_y_pred_gru, squared=False)
r2_score(final_y_real_gru, final_y_pred_gru)
explained_variance_score(final_y_real_gru, final_y_pred_gru)

# Machine Learning Models

In [34]:
def array3dto2d(array):
    return array.reshape(array.shape[0], (array.shape[1]*array.shape[2]))

x_train2d = array3dto2d(x_train)
x_test2d = array3dto2d(x_test)

## Linear Regression

In [35]:
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression().fit(x_train2d, y_train)
linear_regression_prediction = linear_regression_model.predict(x_test2d)

## Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

gradient_boosting_regressor_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(x_train2d, y_train)
gradient_boosting_regressor_prediction = gradient_boosting_regressor_model.predict(x_test2d)

## MultiLayer Perceptron Regressor

In [44]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_model = MLPRegressor(random_state=0).fit(x_train2d, y_train)
mlp_regressor_prediction = mlp_regressor_model.predict(x_test2d)

## Stochastic Gradient Descendent Regressor

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_regressor_model = MultiOutputRegressor(SGDRegressor(random_state=0)).fit(x_train2d, y_train)
sgd_regressor_prediction = sgd_regressor_model.predict(x_test2d)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

random_forest_regressor_model = RandomForestRegressor(random_state=0).fit(x_train2d, y_train)
random_forest_regressor_model.predict(x_test2d)